# Banco de dados

Banco de dados é uma estrutura organizada para armazenar, gerenciar e recuperar informações de forma eficiente. Um termo conhecido é o CRUD (create, read, update e delete). Conheceremos agora um banco de dados para testes, chamado Sqlite3.

In [4]:
# importando a biblioteca para fazer operações em banco de dados sqlite3
import sqlite3
# importando a bibliotecas pandas para transformar dados e tabelas do banco em data frames 
import pandas as pd

# CONNECTION

In [7]:
# criaremos a conexão com nosso banco de dados
# se ele não existir, será criado
conn = sqlite3.connect("database.db")

# CREATE TABLE

In [10]:
# Podemos agora criar tabelas
# criaremos a tabela (table) tasks com campos (fields) id, description e done
conn.execute("""
    CREATE TABLE IF NOT EXISTS tasks
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        description TEXT,
        done INTEGER
    )    
""")
# após executar uma operação é necessário salvá-la
conn.commit()

# CREATE

In [13]:
# criaremos agora um registro (row) na tabela tasks
# o registro terá valor 'Estudar seaborn' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar seaborn', 0);    
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [15]:
# crie um registro (row) na tabela tasks
# o registro terá valor 'Estudar python' para description e 1 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar Python', 1); 
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [17]:
# crie um registro (row) na tabela tasks
# o registro terá valor 'Entregar livro' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Entregar livro', 0);
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

# READ

In [20]:
# podemos agora visualizar todos (*) os registros da (FROM) tabela tasks
results = conn.execute("""
    SELECT *
    FROM tasks
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

(1, 'Estudar seaborn', 0)
(2, 'Estudar Python', 1)
(3, 'Entregar livro', 0)


In [33]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done = 0;
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

('Entregar livro',)


In [35]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0 e description começa com 'Estudar'
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done = 0 AND description LIKE 'Estudar%';
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

# UPDATE

In [40]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo id = 1 fazendo (SET) done = 1
conn.execute("""
    UPDATE tasks
    SET done=1
    WHERE id=1;
""")
# salvando as atualizações
conn.commit()

In [42]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo description = "Estudar seaborn" fazendo (SET) description = "Estudar matplotlib"
conn.execute("""
    UPDATE tasks
    SET description = "Estudar matplotlib"
    WHERE description = "Estudar seaborn";
""")
# salvando as atualizações
conn.commit()

# DELETE

In [45]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn.execute("""
    DELETE FROM tasks
    WHERE id=1
""")
conn.commit()

# Pandas query

Quando se fala em tabelas, uma das bibliotecas mais utilizar é a biblioteca pandas. Para nossa conveniência, o método read_sql_query da biblioteca retorna data frames de consultas SQL.

In [49]:
df = pd.read_sql_query(
    """
       SELECT *
       FROM tasks
    """,
    conn
)
df

,id,description,done
0,2,Estudar Python,1
1,3,Entregar livro,0


# AGRUPAMENTO E AGREGAÇÃO

Existe um paralelo entre os métodos da biblioteca pandas e as queries SQL. Vejamos por exemplo como contar as linhas da tabela tasks onde done = 1

In [53]:
df['done'].value_counts()

done
1    1
0    1
Name: count, dtype: int64

Podemos fazer diretamente usando SQL:

In [56]:
results = conn.execute("""
    SELECT done, COUNT(*) AS done_tasks
    FROM tasks
    GROUP BY done;      
""").fetchall()

for result in results:
    print(result)

(0, 1)
(1, 1)


# Encerrando a conexão

In [59]:
conn.close()

# RELACIONAMENTO

Antes de começar você pode apagar o arquivo database.db, vamos recriá-lo. Se decidir por não apagar, mude o nome do arquivo dessa nova conexão.

In [68]:
# Optei por não excluir, chamando-o de database_2.db
conn = sqlite3.connect("database_2.db")

conn.execute("""
    CREATE TABLE categories 
    (
      id    INTEGER PRIMARY KEY, 
      name  TEXT NOT NULL
    );
""")

conn.execute(
    """
        INSERT INTO categories (name)
        VALUES
          ('casa'),
          ('laser'),
          ('faculdade')
    """
)
conn.commit()

In [70]:
# Observação: removido para essa lição
# conn.close()

A tabela tasks agora terá uma chave estrangeira (FOREIGN KEY) que faz referência ao campo id da tabela categories. Logo estas duas tabelas estão conectadas.

In [73]:
conn.execute("""
    CREATE TABLE tasks 
    (
      id            INTEGER PRIMARY KEY AUTOINCREMENT, 
      description   TEXT NOT NULL,
      done          INTEGER,
      category_id   INTEGER NOT NULL,
      FOREIGN KEY (category_id) REFERENCES categories (id)
    )
""")
conn.commit()

<center><img src="./assets/tasks_categories_relationship.png" width="400px"></center>

# JOIN

Nosso próximo passo é fazer uma querie que junta as duas tabelas anteriores numa só. Antes, vamos inserir alguns dados na tabela tasks.

In [77]:
# insert many
conn.execute(
    """
    INSERT INTO 'tasks' (description, done, category_id)
    VALUES
      ('Assistir Netflix', 1, 2),
      ('Estudar matplotlib', 0, 3),
      ('Estudar seaborn', 0, 3),
      ('Estudar Pandas', 0, 3),
      ('Lavar louça', 0, 1)
    """
)
conn.commit()

Agora sim, juntaremos as tabelas.

In [80]:
tasks_with_categories = conn.execute(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """
).fetchall()

for task in tasks_with_categories:
    print(task)

(5, 'Lavar louça', 0, 'casa')
(2, 'Estudar matplotlib', 0, 'faculdade')
(3, 'Estudar seaborn', 0, 'faculdade')
(4, 'Estudar Pandas', 0, 'faculdade')
(1, 'Assistir Netflix', 1, 'laser')


Usando pandas para ter o resultado como um objeto data frame.

In [83]:
df = pd.read_sql_query(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """,
    conn
)
df

,id,description,done,name
0,5,Lavar louça,0,casa
1,2,Estudar matplotlib,0,faculdade
2,3,Estudar seaborn,0,faculdade
3,4,Estudar Pandas,0,faculdade
4,1,Assistir Netflix,1,laser


# Close connection

In [85]:
conn.close()

# Exercícios

<img src="assets/sqlite-sample-database-color.jpg">

Faça o download do bando SQLite3 no site: https://www.sqlitetutorial.net/sqlite-sample-database/

O arquivo também está na pastas assets, caso não queira fazer o download.

Usando os conhecimentos da aula, resolva os seguinte problemas:

In [206]:
# Extra: conectar com a database
conn = sqlite3.connect("assets/chinook.db")

In [208]:
# Extra: Mostra quais tabelas há dentro da db
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Mostrar os nomes das tabelas
print("Tabelas no banco:")
for table in cursor.fetchall():
    print(table[0])

Tabelas no banco:
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1


1. Fornecer uma consulta que mostre os clientes (apenas seus nomes completos, ID do cliente e país) que não estão nos EUA.

In [211]:
df = pd.read_sql_query(
    """
        SELECT CustomerId, FirstName || ' ' || LastName AS FullName, Country
        FROM customers
        WHERE Country != 'USA'
    """,
    conn
)
df

,CustomerId,FullName,Country
0,1,Luís Gonçalves,Brazil
1,2,Leonie Köhler,Germany
2,3,François Tremblay,Canada
3,4,Bjørn Hansen,Norway
4,5,František Wichterlová,Czech Republic
5,6,Helena Holý,Czech Republic
6,7,Astrid Gruber,Austria
7,8,Daan Peeters,Belgium
8,9,Kara Nielsen,Denmark
9,10,Eduardo Martins,Brazil


2. Fornecer uma consulta que mostre apenas os clientes do Brasil.

In [214]:
df = pd.read_sql_query(
    """
        SELECT CustomerId, FirstName || ' ' || LastName AS FullName, Country
        FROM customers
        WHERE Country = 'Brazil'
    """,
    conn
)
df

,CustomerId,FullName,Country
0,1,Luís Gonçalves,Brazil
1,10,Eduardo Martins,Brazil
2,11,Alexandre Rocha,Brazil
3,12,Roberto Almeida,Brazil
4,13,Fernanda Ramos,Brazil


3. Fornecer uma consulta que mostre as faturas dos clientes que são do Brasil. A tabela resultante deve mostrar o nome completo do cliente, ID da fatura, data da fatura e país de cobrança.

In [217]:
df = pd.read_sql_query(
    """
        SELECT 
            c.FirstName || ' ' || c.LastName AS FullName,
            i.InvoiceId,
            i.InvoiceDate,
            i.BillingCountry
        FROM customers c
        JOIN invoices i ON c.CustomerId = i.CustomerId
        WHERE c.Country = 'Brazil';
    """,
    conn
)
df

,FullName,InvoiceId,InvoiceDate,BillingCountry
0,Luís Gonçalves,98,2010-03-11 00:00:00,Brazil
1,Luís Gonçalves,121,2010-06-13 00:00:00,Brazil
2,Luís Gonçalves,143,2010-09-15 00:00:00,Brazil
3,Luís Gonçalves,195,2011-05-06 00:00:00,Brazil
4,Luís Gonçalves,316,2012-10-27 00:00:00,Brazil
5,Luís Gonçalves,327,2012-12-07 00:00:00,Brazil
6,Luís Gonçalves,382,2013-08-07 00:00:00,Brazil
7,Eduardo Martins,25,2009-04-09 00:00:00,Brazil
8,Eduardo Martins,154,2010-11-14 00:00:00,Brazil
9,Eduardo Martins,177,2011-02-16 00:00:00,Brazil


4. Fornecer uma consulta que mostre apenas os funcionários que são agentes de vendas.

In [220]:
df = pd.read_sql_query(
    """
        SELECT 
            Employeeid,
            FirstName || ' ' || LastName AS FullName,
            Title
        FROM employees
        WHERE Title LIKE "%Sales%";
    """,
    conn
)
df

,EmployeeId,FullName,Title
0,2,Nancy Edwards,Sales Manager
1,3,Jane Peacock,Sales Support Agent
2,4,Margaret Park,Sales Support Agent
3,5,Steve Johnson,Sales Support Agent


5. Fornecer uma consulta que mostre uma lista única de países de cobrança da tabela de faturas.

In [223]:
# Extra: Ordenar por ordem alfabética
df = pd.read_sql_query(
    """
        SELECT DISTINCT BillingCountry
        FROM invoices
        ORDER BY BillingCountry ASC;
    """,
    conn
)
df

,BillingCountry
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Canada
6,Chile
7,Czech Republic
8,Denmark
9,Finland


6. Fornecer uma consulta que mostre as faturas associadas a cada agente de vendas. A tabela resultante deve incluir o nome completo do agente de vendas.

In [226]:
df = pd.read_sql_query(
    """
        SELECT 
            e.FirstName || ' ' || e.LastName AS SalesAgentFullName,
            i.InvoiceId,
            i.InvoiceDate,
            i.BillingCountry
        FROM employees e
        JOIN customers c ON e.EmployeeId = c.SupportRepId
        JOIN invoices i ON c.CustomerId = i.CustomerId
        WHERE e.Title LIKE '%Sales%';
    """,
    conn
)
df

,SalesAgentFullName,InvoiceId,InvoiceDate,BillingCountry
0,Jane Peacock,98,2010-03-11 00:00:00,Brazil
1,Jane Peacock,121,2010-06-13 00:00:00,Brazil
2,Jane Peacock,143,2010-09-15 00:00:00,Brazil
3,Jane Peacock,195,2011-05-06 00:00:00,Brazil
4,Jane Peacock,316,2012-10-27 00:00:00,Brazil
...,...,...,...,...
407,Steve Johnson,88,2010-01-13 00:00:00,Chile
408,Steve Johnson,217,2011-08-20 00:00:00,Chile
409,Steve Johnson,240,2011-11-22 00:00:00,Chile
410,Steve Johnson,262,2012-02-24 00:00:00,Chile


7. Fornecer uma consulta que mostre o valor total da fatura, nome do cliente, país e nome do agente de vendas de todas as faturas e clientes.

In [229]:
df = pd.read_sql_query(
    """
        SELECT 
            i.Total AS InvoiceTotal,
            c.FirstName || ' ' || c.LastName AS CustomerFullName,
            c.Country,
            e.FirstName || ' ' || e.LastName AS SalesAgentFullName
        FROM invoices i
        JOIN customers c ON i.CustomerId = c.CustomerId
        JOIN employees e ON c.SupportRepId = e.EmployeeId;
    """,
    conn
)
df

,InvoiceTotal,CustomerFullName,Country,SalesAgentFullName
0,3.98,Luís Gonçalves,Brazil,Jane Peacock
1,3.96,Luís Gonçalves,Brazil,Jane Peacock
2,5.94,Luís Gonçalves,Brazil,Jane Peacock
3,0.99,Luís Gonçalves,Brazil,Jane Peacock
4,1.98,Luís Gonçalves,Brazil,Jane Peacock
...,...,...,...,...
407,5.94,Puja Srivastava,India,Jane Peacock
408,1.99,Puja Srivastava,India,Jane Peacock
409,1.98,Puja Srivastava,India,Jane Peacock
410,13.86,Puja Srivastava,India,Jane Peacock


8. Quantas faturas houve em 2009 e 2011? Quais são as vendas totais respectivas para cada um desses anos?

In [232]:
# Extra: Faturas e vendas totais em 2009 e 2011
df = pd.read_sql_query(
    """
        SELECT 
            strftime('%Y', InvoiceDate) AS Year,
            COUNT(InvoiceId) AS NumberOfInvoices,
            SUM(Total) AS TotalSales
        FROM invoices
        WHERE strftime('%Y', InvoiceDate) IN ('2009', '2011')
        GROUP BY strftime('%Y', InvoiceDate);
    """,
    conn
)
df

,Year,NumberOfInvoices,TotalSales
0,2009,83,449.46
1,2011,83,469.58


9. Analisando a tabela InvoiceLine, forneça uma consulta que CONTE o número de itens para o ID da fatura 37.

In [235]:
df = pd.read_sql_query(
    """
        SELECT 
            InvoiceId,
            COUNT(InvoiceLineId) AS NumberOfItems
        FROM invoice_items
        WHERE InvoiceId = 37
        GROUP BY InvoiceId;
    """,
    conn
)
df

,InvoiceId,NumberOfItems
0,37,4


10. Analisando a tabela invoice_items, forneça uma consulta que CONTE o número de itens para cada fatura. DICA: AGRUPAR POR

In [238]:
df = pd.read_sql_query(
    """
        SELECT 
            InvoiceId,
            COUNT(InvoiceLineId) AS NumberOfItems
        FROM invoice_items
        GROUP BY InvoiceId;
    """,
    conn
)
df

,InvoiceId,NumberOfItems
0,1,2
1,2,4
2,3,6
3,4,9
4,5,14
...,...,...
407,408,4
408,409,6
409,410,9
410,411,14


11. Fornecer uma consulta que inclua o nome da faixa para cada linha da tabela de itens da fatura.

In [241]:
# Extra: Incluir nome da faixa para cada item da fatura
df = pd.read_sql_query(
    """
        SELECT 
            ii.InvoiceLineId,
            ii.InvoiceId,
            ii.TrackId,
            t.Name AS TrackName,
            ii.UnitPrice,
            ii.Quantity
        FROM invoice_items ii
        JOIN tracks t ON ii.TrackId = t.TrackId;
    """,
    conn
)
df

,InvoiceLineId,InvoiceId,TrackId,TrackName,UnitPrice,Quantity
0,1,1,2,Balls to the Wall,0.99,1
1,2,1,4,Restless and Wild,0.99,1
2,3,2,6,Put The Finger On You,0.99,1
3,4,2,8,Inject The Venom,0.99,1
4,5,2,10,Evil Walks,0.99,1
...,...,...,...,...,...,...
2235,2236,411,3136,Looking For Love,0.99,1
2236,2237,411,3145,Sweet Lady Luck,0.99,1
2237,2238,411,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,0.99,1
2238,2239,411,3163,Samba pras moças,0.99,1


12. Forneça uma consulta que mostre o nome da música e o artista para cada linha da tabela de itens da fatura.

In [244]:
df = pd.read_sql_query(
    """
        SELECT 
            ii.InvoiceLineId,
            ii.InvoiceId,
            ii.TrackId,
            t.Name AS TrackName,
            ar.Name AS ArtistName,
            ii.UnitPrice,
            ii.Quantity
        FROM invoice_items ii
        JOIN tracks t ON ii.TrackId = t.TrackId
        JOIN albums al ON t.AlbumId = al.AlbumId
        JOIN artists ar ON al.ArtistId = ar.ArtistId;
    """,
    conn
)
df

,InvoiceLineId,InvoiceId,TrackId,TrackName,ArtistName,UnitPrice,Quantity
0,1,1,2,Balls to the Wall,Accept,0.99,1
1,2,1,4,Restless and Wild,Accept,0.99,1
2,3,2,6,Put The Finger On You,AC/DC,0.99,1
3,4,2,8,Inject The Venom,AC/DC,0.99,1
4,5,2,10,Evil Walks,AC/DC,0.99,1
...,...,...,...,...,...,...,...
2235,2236,411,3136,Looking For Love,Lenny Kravitz,0.99,1
2236,2237,411,3145,Sweet Lady Luck,Lenny Kravitz,0.99,1
2237,2238,411,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,Zeca Pagodinho,0.99,1
2238,2239,411,3163,Samba pras moças,Zeca Pagodinho,0.99,1


13. Fornecer uma consulta que mostre o número de faturas por país. DICA: AGRUPAR POR

In [247]:
df = pd.read_sql_query(
    """
        SELECT 
            BillingCountry,
            COUNT(InvoiceId) AS NumberOfInvoices
        FROM invoices
        GROUP BY BillingCountry;
    """,
    conn
)
df

,BillingCountry,NumberOfInvoices
0,Argentina,7
1,Australia,7
2,Austria,7
3,Belgium,7
4,Brazil,35
5,Canada,56
6,Chile,7
7,Czech Republic,14
8,Denmark,7
9,Finland,7


14. Forneça uma consulta que mostra o número total de música por playlist. O nome das playlists devem aparecer na tabela resultante.

In [250]:
# Extra: Total de músicas por playlist
df = pd.read_sql_query(
    """
        SELECT 
            p.Name AS PlaylistName,
            COUNT(pt.TrackId) AS TotalTracks
        FROM playlists p
        LEFT JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId
        GROUP BY p.Name;
    """,
    conn
)
df

,PlaylistName,TotalTracks
0,90’s Music,1477
1,Audiobooks,0
2,Brazilian Music,39
3,Classical,75
4,Classical 101 - Deep Cuts,25
5,Classical 101 - Next Steps,25
6,Classical 101 - The Basics,25
7,Grunge,15
8,Heavy Metal Classic,26
9,Movies,0


15.Forneça uma consulta que mostra todos os nomes das músicas, mas não mostra ids. A tabela resultante deve incluir o nome do álbum, tipo de media e gênero.

In [253]:
# Extra: Nomes das músicas com álbum, tipo de mídia e gênero
df = pd.read_sql_query(
    """
        SELECT 
            t.Name AS TrackName,
            al.Title AS AlbumName,
            mt.Name AS MediaType,
            g.Name AS Genre
        FROM tracks t
        JOIN albums al ON t.AlbumId = al.AlbumId
        JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId
        JOIN genres g ON t.GenreId = g.GenreId;
    """,
    conn
)
df

,TrackName,AlbumName,MediaType,Genre
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,MPEG audio file,Rock
1,Balls to the Wall,Balls to the Wall,Protected AAC audio file,Rock
2,Fast As a Shark,Restless and Wild,Protected AAC audio file,Rock
3,Restless and Wild,Restless and Wild,Protected AAC audio file,Rock
4,Princess of the Dawn,Restless and Wild,Protected AAC audio file,Rock
...,...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome,Protected AAC audio file,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,Protected AAC audio file,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo,Protected AAC audio file,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music,Protected AAC audio file,Classical


16. Forneça uma consulta que mostra todas as faturas por id e a contagem de itens por fatura.

In [256]:
df = pd.read_sql_query(
    """
        SELECT 
            i.InvoiceId,
            COUNT(ii.InvoiceLineId) AS NumberOfItems
        FROM invoices i
        JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
        GROUP BY i.InvoiceId
        ORDER BY i.InvoiceId;
    """,
    conn
)
df

,InvoiceId,NumberOfItems
0,1,2
1,2,4
2,3,6
3,4,9
4,5,14
...,...,...
407,408,4
408,409,6
409,410,9
410,411,14


17. Fornecer uma consulta que mostro o número total de vendas de cada vendedor.

In [259]:
df = pd.read_sql_query(
    """
        SELECT 
            e.EmployeeId,
            e.FirstName || ' ' || e.LastName AS SalesRepName,
            COUNT(i.InvoiceId) AS TotalSales
        FROM employees e
        JOIN customers c ON e.EmployeeId = c.SupportRepId
        JOIN invoices i ON c.CustomerId = i.CustomerId
        GROUP BY e.EmployeeId
        ORDER BY TotalSales DESC;
    """,
    conn
)
df

,EmployeeId,SalesRepName,TotalSales
0,3,Jane Peacock,146
1,4,Margaret Park,140
2,5,Steve Johnson,126


18. Qual vendedor (sales agent) mais vendeu em 2009?

In [262]:
df = pd.read_sql_query(
    """
        SELECT 
            e.EmployeeId,
            e.FirstName || ' ' || e.LastName AS SalesRepName,
            SUM(i.Total) AS TotalSales2009
        FROM employees e
        JOIN customers c ON e.EmployeeId = c.SupportRepId
        JOIN invoices i ON c.CustomerId = i.CustomerId
        WHERE strftime('%Y', i.InvoiceDate) = '2009'
        GROUP BY e.EmployeeId
        ORDER BY TotalSales2009 DESC
        LIMIT 1;
    """,
    conn
)
df


,EmployeeId,SalesRepName,TotalSales2009
0,5,Steve Johnson,164.34


19. Qual vendedor (sales agent) mais vendeu em 2010?

In [265]:
df = pd.read_sql_query(
    """
        SELECT 
            e.EmployeeId,
            e.FirstName || ' ' || e.LastName AS SalesRepName,
            SUM(i.Total) AS TotalSales2010
        FROM employees e
        JOIN customers c ON e.EmployeeId = c.SupportRepId
        JOIN invoices i ON c.CustomerId = i.CustomerId
        WHERE strftime('%Y', i.InvoiceDate) = '2010'
        GROUP BY e.EmployeeId
        ORDER BY TotalSales2010 DESC
        LIMIT 1;
    """,
    conn
)
df


,EmployeeId,SalesRepName,TotalSales2010
0,3,Jane Peacock,221.92


20. Qual vendedor (sales agent) mais vendeu considerando toda base de dados?

In [268]:
df = pd.read_sql_query(
    """
    SELECT 
        e.EmployeeId,
        e.FirstName || ' ' || e.LastName AS SalesRepName,
        SUM(i.Total) AS TotalSales
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    JOIN invoices i ON c.CustomerId = i.CustomerId
    GROUP BY e.EmployeeId
    ORDER BY TotalSales DESC
    LIMIT 1;
    """,
    conn
)
df

,EmployeeId,SalesRepName,TotalSales
0,3,Jane Peacock,833.04


21. Forneça uma consulta que mostra o número de clientes inscritos para cada vendedor (sales agente).

In [271]:
df = pd.read_sql_query(
    """
    SELECT 
        e.EmployeeId,
        e.FirstName || ' ' || e.LastName AS SalesRepName,
        COUNT(c.CustomerId) AS NumberOfCustomers
    FROM employees e
    LEFT JOIN customers c ON e.EmployeeId = c.SupportRepId
    WHERE e.Title LIKE '%Sales Support Agent%'
    GROUP BY e.EmployeeId
    ORDER BY NumberOfCustomers DESC;
    """,
    conn
)
df

,EmployeeId,SalesRepName,NumberOfCustomers
0,3,Jane Peacock,21
1,4,Margaret Park,20
2,5,Steve Johnson,18


22. Forneça uma consulta que mostra o total de vendas por país.

In [274]:
df = pd.read_sql_query(
    """
    SELECT 
        i.BillingCountry AS Country,
        SUM(i.Total) AS TotalSales
    FROM invoices i
    GROUP BY i.BillingCountry
    ORDER BY TotalSales DESC;
    """,
    conn
)
df

,Country,TotalSales
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


23. De qual país os clientes mais gastaram?

In [277]:
df = pd.read_sql_query(
    """
    SELECT 
        c.Country AS CustomerCountry,
        SUM(ii.UnitPrice * ii.Quantity) AS TotalSpent
    FROM customers c
    JOIN invoices i ON c.CustomerId = i.CustomerId
    JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
    GROUP BY c.Country
    ORDER BY TotalSpent DESC
    LIMIT 1;
    """,
    conn
)
df

,CustomerCountry,TotalSpent
0,USA,523.06


24. Forneça uma consulta que mostra a faixa mais comprada de 2013.

In [280]:
# Extra: mostra outras 5 mais comparadas pra verificação
df = pd.read_sql_query(
    """
    SELECT 
        t.TrackId,
        t.Name AS TrackName,
        SUM(ii.Quantity) AS TotalPurchased,
        a.Title AS Album,
        ar.Name AS Artist
    FROM invoice_items ii
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    JOIN tracks t ON ii.TrackId = t.TrackId
    JOIN albums a ON t.AlbumId = a.AlbumId
    JOIN artists ar ON a.ArtistId = ar.ArtistId
    WHERE strftime('%Y', i.InvoiceDate) = '2013'
    GROUP BY t.TrackId
    ORDER BY TotalPurchased DESC
    LIMIT 5; 
    """,
    conn
)
df

,TrackId,TrackName,TotalPurchased,Album,Artist
0,3177,Hot Girl,1,"The Office, Season 1",The Office
1,3163,Samba pras moças,1,Ao Vivo [IMPORT],Zeca Pagodinho
2,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,1,Ao Vivo [IMPORT],Zeca Pagodinho
3,3145,Sweet Lady Luck,1,Greatest Hits,Lenny Kravitz
4,3136,Looking For Love,1,Greatest Hits,Lenny Kravitz


25. Forneça uma consulta que mostra as top 5 faixas mais compradas de todos os tempos.

In [283]:
df = pd.read_sql_query(
    """
    SELECT 
        t.TrackId,
        t.Name AS TrackName,
        ar.Name AS Artist,
        a.Title AS Album,
        SUM(ii.Quantity) AS TotalPurchased
    FROM invoice_items ii
    JOIN tracks t ON ii.TrackId = t.TrackId
    JOIN albums a ON t.AlbumId = a.AlbumId
    JOIN artists ar ON a.ArtistId = ar.ArtistId
    GROUP BY t.TrackId
    ORDER BY TotalPurchased DESC
    LIMIT 5;
    """,
    conn
)
df

,TrackId,TrackName,Artist,Album,TotalPurchased
0,2,Balls to the Wall,Accept,Balls to the Wall,2
1,8,Inject The Venom,AC/DC,For Those About To Rock We Salute You,2
2,9,Snowballed,AC/DC,For Those About To Rock We Salute You,2
3,20,Overdose,AC/DC,Let There Be Rock,2
4,32,Deuces Are Wild,Aerosmith,Big Ones,2


26. Forneça uma consulta que mostra os top 3 artistas mais vendidos de toda base de dados.

In [286]:
df = pd.read_sql_query(
    """
    SELECT 
        ar.ArtistId,
        ar.Name AS ArtistName,
        SUM(ii.Quantity) AS TotalTracksSold
    FROM artists ar
    JOIN albums a ON ar.ArtistId = a.ArtistId
    JOIN tracks t ON a.AlbumId = t.AlbumId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY ar.ArtistId
    ORDER BY TotalTracksSold DESC
    LIMIT 3;
    """,
    conn
)
df

,ArtistId,ArtistName,TotalTracksSold
0,90,Iron Maiden,140
1,150,U2,107
2,50,Metallica,91


27. Forneça uma query que mostra qual o tipo de media mais comprado.

In [289]:
df = pd.read_sql_query(
    """
    SELECT 
        mt.MediaTypeId,
        mt.Name AS MediaType,
        SUM(ii.Quantity) AS TotalPurchased
    FROM media_types mt
    JOIN tracks t ON mt.MediaTypeId = t.MediaTypeId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY mt.MediaTypeId
    ORDER BY TotalPurchased DESC
    LIMIT 1;
    """,
    conn
)
df

,MediaTypeId,MediaType,TotalPurchased
0,1,MPEG audio file,1976


### Referência para os exercícios:

http://www.berkut.mk.ua/download/pdf/db/task_vesna.pdf

In [293]:
# Extra: Fechar a database
conn.close()